# Finding mismatched points

In [1]:
%matplotlib inline

import sys
import os
import rasterio
import xarray
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import plot_confusion_matrix, f1_score  
from deafrica_tools.plotting import map_shapefile,display_map, rgb
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.datahandling import wofs_fuser, mostcommon_crs,load_ard,deepcopy
from deafrica_tools.dask import create_local_dask_cluster

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
old_wofs = pd.read_csv('../Results/WOfS_Assessment/Beta/Point_Based/ValidPoints_Per_AEZ/Africa_ValidationPoints.csv')
new_wofs = pd.read_csv('../Results/WOfS_Assessment/wofs_ls/Africa_Combined_wofs_ls.csv')


In [5]:
old_wofs

,Unnamed: 0,PLOT_ID,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION
0,0,137483175.0,30.463813,-26.653807,Open water - freshwater,1,1,1.0,2.0,1
1,5,137483175.0,30.463813,-26.653807,Open water - freshwater,5,1,1.0,1.0,1
2,6,137483175.0,30.463813,-26.653807,Open water - freshwater,6,1,1.0,3.0,1
3,7,137483175.0,30.463813,-26.653807,Open water - freshwater,7,1,1.0,1.0,1
4,9,137483175.0,30.463813,-26.653807,Open water - freshwater,9,1,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...
12143,40086,137712352.0,6.026038,4.435646,Open water - marine,12,1,1.0,1.0,1
12144,40095,137712353.0,5.840416,4.426212,Open water - marine,1,1,1.0,1.0,1
12145,40107,137712354.0,6.631720,4.347916,Open water - marine,1,1,1.0,1.0,1
12146,40109,137712354.0,6.631720,4.347916,Open water - marine,3,1,1.0,1.0,1


In [7]:
new_wofs.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'PLOT_ID', 'LON', 'LAT', 'FLAGGED',
       'ANALYSES', 'SENTINEL2Y', 'STARTDATE', 'ENDDATE', 'WATER', 'NO_WATER',
       'BAD_IMAGE', 'NOT_SURE', 'CLASS', 'COMMENT', 'MONTH', 'WATERFLAG',
       'geometry', 'CLASS_WET', 'CLEAR_OBS'],
      dtype='object')

In [8]:
new_wofs = new_wofs.drop(['Unnamed: 0', 'Unnamed: 0.1', 'FLAGGED', 'ANALYSES','SENTINEL2Y','STARTDATE', 'ENDDATE', 'WATER', 'NO_WATER', 'BAD_IMAGE', 'NOT_SURE','COMMENT','geometry'], axis=1)

In [9]:
new_wofs

,PLOT_ID,LON,LAT,CLASS,MONTH,WATERFLAG,CLASS_WET,CLEAR_OBS
0,137711631.0,17.782114,7.802986,Open water - freshwater,8,0,1.0,1.0
1,137711631.0,17.782114,7.802986,Open water - freshwater,10,0,1.0,2.0
2,137711631.0,17.782114,7.802986,Open water - freshwater,1,1,NaN,NaN
3,137711631.0,17.782114,7.802986,Open water - freshwater,2,1,1.0,2.0
4,137711631.0,17.782114,7.802986,Open water - freshwater,3,1,0.0,2.0
...,...,...,...,...,...,...,...,...
40126,137482804.0,13.838962,-9.369115,Open water - freshwater,11,0,0.0,0.0
40127,137482804.0,13.838962,-9.369115,Open water - freshwater,11,2,0.0,0.0
40128,137482804.0,13.838962,-9.369115,Open water - freshwater,12,0,0.0,0.0
40129,137482804.0,13.838962,-9.369115,Open water - freshwater,12,2,0.0,0.0


In [10]:
#Remove the duplicated plot IDs which means those that are labeled for similar month as 0, 1, 2  or 3.
Duplicate = new_wofs.duplicated(['LAT', 'LON','MONTH'], keep=False)
new_wofs = new_wofs[Duplicate==False]

In [11]:
new_wofs

,PLOT_ID,LON,LAT,CLASS,MONTH,WATERFLAG,CLASS_WET,CLEAR_OBS
138,137711642.0,23.061932,5.240099,Forest/woodlands,10,0,0.0,1.0
193,137711646.0,23.879043,4.907491,Open water - freshwater,10,2,0.0,0.0
1387,137711746.0,16.223136,-2.101133,Open water - freshwater,3,2,0.0,0.0
1394,137711747.0,18.175745,-2.195967,Open water - freshwater,2,1,0.0,0.0
1491,137711755.0,16.208833,-2.713546,Open water - freshwater,6,1,1.0,1.0
...,...,...,...,...,...,...,...,...
39901,137482794.0,13.126633,-7.844516,Open water - marine,6,2,0.0,0.0
39902,137482794.0,13.126633,-7.844516,Open water - marine,7,2,0.0,0.0
39903,137482794.0,13.126633,-7.844516,Open water - marine,8,2,0.0,0.0
39904,137482794.0,13.126633,-7.844516,Open water - marine,10,2,0.0,0.0


In [12]:
#Counting the number of duplicates in validation points in AEZ  
countduplicate = new_wofs.groupby('PLOT_ID',as_index=False,sort=False).last()
countduplicate

,PLOT_ID,LON,LAT,CLASS,MONTH,WATERFLAG,CLASS_WET,CLEAR_OBS
0,137711642.0,23.061932,5.240099,Forest/woodlands,10,0,0.0,1.0
1,137711646.0,23.879043,4.907491,Open water - freshwater,10,2,0.0,0.0
2,137711746.0,16.223136,-2.101133,Open water - freshwater,3,2,0.0,0.0
3,137711747.0,18.175745,-2.195967,Open water - freshwater,2,1,0.0,0.0
4,137711755.0,16.208833,-2.713546,Open water - freshwater,6,1,1.0,1.0
...,...,...,...,...,...,...,...,...
2488,137482620.0,17.782114,7.802986,Open water - freshwater,11,1,1.0,2.0
2489,137482691.0,25.103465,0.524176,Open water - freshwater,14,1,NaN,NaN
2490,137482712.0,10.421585,-0.737478,Open water - freshwater,7,2,0.0,0.0
2491,137482720.0,16.863641,-1.227621,Open water - freshwater,3,1,0.0,1.0


In [13]:
new_wofs = new_wofs.rename(columns={'WATERFLAG':'ACTUAL'})

In [19]:
#Filter out those rows that are labeled more than 1 or there is no clear WOfS/SCL observations  
indexNames = new_wofs[(new_wofs['ACTUAL'] > 1) | (new_wofs['CLEAR_OBS']==0.0) | (new_wofs['CLEAR_OBS'].isna())].index
indexNames
new_wofs.drop(indexNames, inplace=True)

In [20]:
new_wofs

,PLOT_ID,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS
138,137711642.0,23.061932,5.240099,Forest/woodlands,10,0,0.0,1.0
1491,137711755.0,16.208833,-2.713546,Open water - freshwater,6,1,1.0,1.0
1918,137711790.0,22.334056,-5.938465,Cultivated (Cropland/ Plantation),6,1,1.0,2.0
2097,137711805.0,13.126633,-7.844516,Open water - marine,5,1,1.0,1.0
2098,137711805.0,13.126633,-7.844516,Open water - marine,9,1,1.0,1.0
...,...,...,...,...,...,...,...,...
36221,137483527.0,8.750363,4.708335,Open water - freshwater,12,1,0.0,1.0
36237,137483528.0,9.411390,4.659262,Open water - freshwater,2,1,1.0,1.0
36260,137483529.0,8.870069,4.549805,Open water - freshwater,2,1,0.0,1.0
36276,137482620.0,17.782114,7.802986,Open water - freshwater,11,1,1.0,2.0


In [21]:
old_wofs

,Unnamed: 0,PLOT_ID,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION
0,0,137483175.0,30.463813,-26.653807,Open water - freshwater,1,1,1.0,2.0,1
1,5,137483175.0,30.463813,-26.653807,Open water - freshwater,5,1,1.0,1.0,1
2,6,137483175.0,30.463813,-26.653807,Open water - freshwater,6,1,1.0,3.0,1
3,7,137483175.0,30.463813,-26.653807,Open water - freshwater,7,1,1.0,1.0,1
4,9,137483175.0,30.463813,-26.653807,Open water - freshwater,9,1,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...
12143,40086,137712352.0,6.026038,4.435646,Open water - marine,12,1,1.0,1.0,1
12144,40095,137712353.0,5.840416,4.426212,Open water - marine,1,1,1.0,1.0,1
12145,40107,137712354.0,6.631720,4.347916,Open water - marine,1,1,1.0,1.0,1
12146,40109,137712354.0,6.631720,4.347916,Open water - marine,3,1,1.0,1.0,1


In [22]:
new_wofs

,PLOT_ID,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS
138,137711642.0,23.061932,5.240099,Forest/woodlands,10,0,0.0,1.0
1491,137711755.0,16.208833,-2.713546,Open water - freshwater,6,1,1.0,1.0
1918,137711790.0,22.334056,-5.938465,Cultivated (Cropland/ Plantation),6,1,1.0,2.0
2097,137711805.0,13.126633,-7.844516,Open water - marine,5,1,1.0,1.0
2098,137711805.0,13.126633,-7.844516,Open water - marine,9,1,1.0,1.0
...,...,...,...,...,...,...,...,...
36221,137483527.0,8.750363,4.708335,Open water - freshwater,12,1,0.0,1.0
36237,137483528.0,9.411390,4.659262,Open water - freshwater,2,1,1.0,1.0
36260,137483529.0,8.870069,4.549805,Open water - freshwater,2,1,0.0,1.0
36276,137482620.0,17.782114,7.802986,Open water - freshwater,11,1,1.0,2.0


In [32]:
cross_match = new_wofs.merge(old_wofs, on=['PLOT_ID', 'MONTH'])
cross_match

,PLOT_ID,LON_x,LAT_x,CLASS_x,MONTH,ACTUAL_x,CLASS_WET_x,CLEAR_OBS_x,Unnamed: 0,LON_y,LAT_y,CLASS_y,ACTUAL_y,CLASS_WET_y,CLEAR_OBS_y,PREDICTION
0,137711642.0,23.061932,5.240099,Forest/woodlands,10,0,0.0,1.0,31545,23.061932,5.240099,Forest/woodlands,0,0.0,1.0,0
1,137711755.0,16.208833,-2.713546,Open water - freshwater,6,1,1.0,1.0,32898,16.208833,-2.713546,Open water - freshwater,1,1.0,1.0,1
2,137711790.0,22.334056,-5.938465,Cultivated (Cropland/ Plantation),6,1,1.0,2.0,33325,22.334056,-5.938465,Cultivated (Cropland/ Plantation),1,1.0,1.0,1
3,137711805.0,13.126633,-7.844516,Open water - marine,5,1,1.0,1.0,33504,13.126633,-7.844516,Open water - marine,1,1.0,1.0,1
4,137711805.0,13.126633,-7.844516,Open water - marine,9,1,1.0,1.0,33505,13.126633,-7.844516,Open water - marine,1,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,137483527.0,8.750363,4.708335,Open water - freshwater,2,1,0.0,1.0,9922,8.750363,4.708335,Open water - freshwater,1,1.0,1.0,1
9705,137483527.0,8.750363,4.708335,Open water - freshwater,12,1,0.0,1.0,9925,8.750363,4.708335,Open water - freshwater,1,0.0,1.0,0
9706,137483528.0,9.411390,4.659262,Open water - freshwater,2,1,1.0,1.0,9941,9.411390,4.659262,Open water - freshwater,1,1.0,1.0,1
9707,137483529.0,8.870069,4.549805,Open water - freshwater,2,1,0.0,1.0,9964,8.870069,4.549805,Open water - freshwater,1,1.0,1.0,1


In [33]:
cross_match['different']=cross_match.CLASS_WET_x != cross_match.CLASS_WET_y

In [34]:
cross_match['different']

0       False
1       False
2       False
3       False
4       False
        ...  
9704     True
9705    False
9706    False
9707     True
9708    False
Name: different, Length: 9709, dtype: bool

In [35]:
indexNames = cross_match[(cross_match['different'] == False)].index
indexNames
cross_match.drop(indexNames, inplace=True)

In [36]:
cross_match

,PLOT_ID,LON_x,LAT_x,CLASS_x,MONTH,ACTUAL_x,CLASS_WET_x,CLEAR_OBS_x,Unnamed: 0,LON_y,LAT_y,CLASS_y,ACTUAL_y,CLASS_WET_y,CLEAR_OBS_y,PREDICTION,different
44,137711831.0,17.793307,-11.063076,Cultivated (Cropland/ Plantation),7,0,1.0,2.0,33817,17.793307,-11.063076,Cultivated (Cropland/ Plantation),0,0.0,1.0,0,True
88,137711847.0,16.411867,-13.335214,Open water - freshwater,7,1,1.0,2.0,34008,16.411867,-13.335214,Open water - freshwater,1,0.0,1.0,0,True
90,137711847.0,16.411867,-13.335214,Open water - freshwater,9,1,0.0,1.0,34010,16.411867,-13.335214,Open water - freshwater,1,1.0,1.0,1,True
110,137711852.0,13.189751,-15.253757,Forest/woodlands,6,0,0.0,1.0,34069,13.189751,-15.253757,Forest/woodlands,0,1.0,1.0,1,True
200,137711875.0,28.625624,-11.926044,Open water - freshwater,6,1,0.0,1.0,34342,28.625624,-11.926044,Open water - freshwater,1,1.0,1.0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9602,137483433.0,1.785642,6.629410,Open water - freshwater,5,1,1.0,2.0,7998,1.785642,6.629410,Open water - freshwater,1,0.0,2.0,0,True
9632,137483503.0,-6.939536,4.628830,Open water - marine,4,1,0.0,1.0,9528,-6.939536,4.628830,Open water - marine,1,1.0,1.0,1,True
9637,137483508.0,7.241133,4.496497,Open water - freshwater,4,1,1.0,1.0,9633,7.241133,4.496497,Open water - freshwater,1,0.0,1.0,0,True
9704,137483527.0,8.750363,4.708335,Open water - freshwater,2,1,0.0,1.0,9922,8.750363,4.708335,Open water - freshwater,1,1.0,1.0,1,True


In [39]:
cross_match.to_csv('../Results/WOfS_Assessment/wofs_ls/points_different_wofs_ls.csv')

In [41]:
gdf = gpd.GeoDataFrame(cross_match, geometry=gpd.points_from_xy(cross_match.LON_x, cross_match.LAT_x))

In [42]:
gdf.to_file('../Results/WOfS_Assessment/wofs_ls/points_different_wofs_ls.shp')

## Joining files for Meghan

Using the cross_validation_results.csv as well as the results from new wofs and old wofs and Meghan's results we will join all of these together to make a big table so that it can be analysed for the paper.

In [2]:
# first load in Meghan's for the new wofs and match it to one for the old wofs
old_wofs = pd.read_csv('../Results/WOfS_Assessment/Point_Based/AGRHYMET_CV_Project-1_with_id.csv')
new_wofs = pd.read_csv('../Results/WOfS_Assessment/Point_Based/Meghan_valid_points.csv')

In [5]:
old_wofs

,Unnamed: 0,lat,lon,month,plot_id,pl_plotid,class,waterflag,geometry,CLASS_WET,CLEAR_OBS
0,0,7.103984,13.880937,1,138282925.0,1.0,Herbaceous/Grassland,0,POINT (1339319.999718883 904019.9999515081),0.0,1.0
1,1,7.103984,13.880937,4,138282925.0,1.0,Herbaceous/Grassland,0,POINT (1339319.999718883 904019.9999515081),0.0,0.0
2,2,7.103984,13.880937,5,138282925.0,1.0,Herbaceous/Grassland,0,POINT (1339319.999718883 904019.9999515081),0.0,0.0
3,3,7.103984,13.880937,2,138282925.0,1.0,Herbaceous/Grassland,2,POINT (1339319.999718883 904019.9999515081),0.0,1.0
4,4,7.103984,13.880937,3,138282925.0,1.0,Herbaceous/Grassland,2,POINT (1339319.999718883 904019.9999515081),0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
944,944,-5.424299,29.747131,3,138283004.0,80.0,Open water - freshwater,2,POINT (2870189.999993738 -690990.0000399862),0.0,0.0
945,945,-5.424299,29.747131,9,138283004.0,80.0,Open water - freshwater,2,POINT (2870189.999993738 -690990.0000399862),1.0,1.0
946,946,-5.424299,29.747131,10,138283004.0,80.0,Open water - freshwater,2,POINT (2870189.999993738 -690990.0000399862),NaN,NaN
947,947,-5.424299,29.747131,11,138283004.0,80.0,Open water - freshwater,2,POINT (2870189.999993738 -690990.0000399862),NaN,NaN


In [6]:
new_wofs

,Unnamed: 0,lat,lon,month,plot_id,pl_plotid,class,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION
0,3,7.103984,13.880937,10,139368549.0,1.0,Forest/woodlands,0,0.0,1.0,0
1,8,-16.394333,18.989746,1,139368550.0,2.0,Open water - freshwater,1,1.0,1.0,1
2,10,-16.394333,18.989746,5,139368550.0,2.0,Open water - freshwater,1,1.0,1.0,1
3,11,-16.394333,18.989746,6,139368550.0,2.0,Open water - freshwater,1,1.0,2.0,1
4,12,-16.394333,18.989746,7,139368550.0,2.0,Open water - freshwater,1,1.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...
272,871,-2.393420,33.521761,10,139368627.0,79.0,Open water - freshwater,1,1.0,1.0,1
273,873,-2.393420,33.521761,12,139368627.0,79.0,Open water - freshwater,1,1.0,1.0,1
274,880,-5.424299,29.747131,6,139368628.0,80.0,Open water - freshwater,1,1.0,1.0,1
275,881,-5.424299,29.747131,7,139368628.0,80.0,Open water - freshwater,1,1.0,2.0,1


In [10]:
Meghan = new_wofs.merge(old_wofs, on=['lat', 'lon', 'month'])
Meghan[:3]

,Unnamed: 0_x,lat,lon,month,plot_id_x,pl_plotid_x,class_x,ACTUAL,CLASS_WET_x,CLEAR_OBS_x,PREDICTION,Unnamed: 0_y,plot_id_y,pl_plotid_y,class_y,waterflag,geometry,CLASS_WET_y,CLEAR_OBS_y
0,55,-25.97481,29.6125,1,139368554.0,6.0,Open water - freshwater,1,0.0,1.0,0,63,138282930.0,6.0,Open water - Constructed (e.g. aquaculture),2,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
1,57,-25.97481,29.6125,5,139368554.0,6.0,Open water - freshwater,1,0.0,2.0,0,59,138282930.0,6.0,Open water - Constructed (e.g. aquaculture),1,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
2,58,-25.97481,29.6125,7,139368554.0,6.0,Open water - freshwater,1,0.0,1.0,0,60,138282930.0,6.0,Open water - Constructed (e.g. aquaculture),1,POINT (2857199.999980968 -3204089.999644605),0.0,1.0


In [11]:
Meghan.columns

Index(['Unnamed: 0_x', 'lat', 'lon', 'month', 'plot_id_x', 'pl_plotid_x',
       'class_x', 'ACTUAL', 'CLASS_WET_x', 'CLEAR_OBS_x', 'PREDICTION',
       'Unnamed: 0_y', 'plot_id_y', 'pl_plotid_y', 'class_y', 'waterflag',
       'geometry', 'CLASS_WET_y', 'CLEAR_OBS_y'],
      dtype='object')

In [16]:
#Meghan = Meghan.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'pl_plotid_x', 'pl_plotid_y','plot_id_x','plot_id_y'], axis=1)
Meghan = Meghan.drop(['class_y', 'waterflag'],axis=1)

In [18]:
Meghan[:3]

,lat,lon,month,class_x,ACTUAL,CLASS_WET_x,CLEAR_OBS_x,PREDICTION,geometry,CLASS_WET_y,CLEAR_OBS_y
0,-25.97481,29.6125,1,Open water - freshwater,1,0.0,1.0,0,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
1,-25.97481,29.6125,5,Open water - freshwater,1,0.0,2.0,0,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
2,-25.97481,29.6125,7,Open water - freshwater,1,0.0,1.0,0,POINT (2857199.999980968 -3204089.999644605),0.0,1.0


In [19]:
Meghan = Meghan.rename(columns={'PREDICTION':'new_wofs'})

In [20]:
Meghan = Meghan.rename(columns={'ACTUAL':'waterflag_Meghan'})

In [21]:
Meghan = Meghan.rename(columns={'CLASS_WET_x':'new_wofs_wet'})

In [22]:
Meghan = Meghan.rename(columns={'CLASS_WET_y':'old_wofs_wet'})

In [23]:
Meghan = Meghan.rename(columns={'CLEAR_OBS_x':'new_wofs_clear'})

In [24]:
Meghan = Meghan.rename(columns={'CLEAR_OBS_y':'old_wofs_clear'})

In [29]:
Meghan

,lat,lon,month,class_x,waterflag_Meghan,new_wofs_wet,new_wofs_clear,new_wofs,geometry,old_wofs_wet,old_wofs_clear
0,-25.974810,29.612500,1,Open water - freshwater,1,0.0,1.0,0,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
1,-25.974810,29.612500,5,Open water - freshwater,1,0.0,2.0,0,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
2,-25.974810,29.612500,7,Open water - freshwater,1,0.0,1.0,0,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
3,-25.974810,29.612500,9,Open water - freshwater,1,1.0,1.0,1,POINT (2857199.999980968 -3204089.999644605),1.0,1.0
4,-25.974810,29.612500,12,Open water - freshwater,1,0.0,1.0,0,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
5,9.490169,11.964706,2,Open water - freshwater,1,0.0,1.0,0,POINT (1154430.000357153 1205310.000009574),0.0,1.0
6,9.490169,11.964706,6,Open water - freshwater,1,0.0,1.0,0,POINT (1154430.000357153 1205310.000009574),0.0,1.0
7,9.490169,11.964706,11,Open water - freshwater,1,1.0,1.0,1,POINT (1154430.000357153 1205310.000009574),NaN,NaN
8,9.490169,11.964706,12,Open water - freshwater,1,1.0,1.0,1,POINT (1154430.000357153 1205310.000009574),1.0,1.0
9,-29.354791,29.923736,1,Open water - freshwater,1,0.0,1.0,0,POINT (2887230.000041685 -3587039.999866918),1.0,3.0


In [27]:
# now read in the big csv
cross_val = pd.read_csv('../Results/WOfS_Assessment/Point_Based/cross_validation_results.csv')
cross_val[:3]

,Unnamed: 0,lat,lon,month,waterflag,waterflagAFR,waterflagAFR.1,waterflagAGR,waterflagAGR.1,waterflagRCM,waterflagRCM.1,waterflagAFR.2,waterflagAFR.3,waterflag_mode,agree_percent
0,0,7.103984,13.880937,1,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,100.0
1,1,7.103984,13.880937,2,0.0,NaN,NaN,2.0,2.0,NaN,0.0,NaN,NaN,0.0,50.0
2,2,7.103984,13.880937,4,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,100.0


In [28]:
cross_Meghan = cross_val.merge(Meghan, on=['lat', 'lon', 'month'])
cross_Meghan[:3]

,Unnamed: 0,lat,lon,month,waterflag,waterflagAFR,waterflagAFR.1,waterflagAGR,waterflagAGR.1,waterflagRCM,...,waterflag_mode,agree_percent,class_x,waterflag_Meghan,new_wofs_wet,new_wofs_clear,new_wofs,geometry,old_wofs_wet,old_wofs_clear
0,60,-25.97481,29.6125,5,1.0,NaN,1.0,1.0,1.0,NaN,...,1.0,85.714286,Open water - freshwater,1,0.0,2.0,0,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
1,62,-25.97481,29.6125,7,1.0,1.0,1.0,1.0,1.0,NaN,...,1.0,100.000000,Open water - freshwater,1,0.0,1.0,0,POINT (2857199.999980968 -3204089.999644605),0.0,1.0
2,64,-25.97481,29.6125,9,1.0,1.0,1.0,1.0,1.0,NaN,...,1.0,100.000000,Open water - freshwater,1,1.0,1.0,1,POINT (2857199.999980968 -3204089.999644605),1.0,1.0


In [30]:
new_wofs

,Unnamed: 0,lat,lon,month,plot_id,pl_plotid,class,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION
0,3,7.103984,13.880937,10,139368549.0,1.0,Forest/woodlands,0,0.0,1.0,0
1,8,-16.394333,18.989746,1,139368550.0,2.0,Open water - freshwater,1,1.0,1.0,1
2,10,-16.394333,18.989746,5,139368550.0,2.0,Open water - freshwater,1,1.0,1.0,1
3,11,-16.394333,18.989746,6,139368550.0,2.0,Open water - freshwater,1,1.0,2.0,1
4,12,-16.394333,18.989746,7,139368550.0,2.0,Open water - freshwater,1,1.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...
272,871,-2.393420,33.521761,10,139368627.0,79.0,Open water - freshwater,1,1.0,1.0,1
273,873,-2.393420,33.521761,12,139368627.0,79.0,Open water - freshwater,1,1.0,1.0,1
274,880,-5.424299,29.747131,6,139368628.0,80.0,Open water - freshwater,1,1.0,1.0,1
275,881,-5.424299,29.747131,7,139368628.0,80.0,Open water - freshwater,1,1.0,2.0,1


In [31]:
old_wofs

,Unnamed: 0,lat,lon,month,plot_id,pl_plotid,class,waterflag,geometry,CLASS_WET,CLEAR_OBS
0,0,7.103984,13.880937,1,138282925.0,1.0,Herbaceous/Grassland,0,POINT (1339319.999718883 904019.9999515081),0.0,1.0
1,1,7.103984,13.880937,4,138282925.0,1.0,Herbaceous/Grassland,0,POINT (1339319.999718883 904019.9999515081),0.0,0.0
2,2,7.103984,13.880937,5,138282925.0,1.0,Herbaceous/Grassland,0,POINT (1339319.999718883 904019.9999515081),0.0,0.0
3,3,7.103984,13.880937,2,138282925.0,1.0,Herbaceous/Grassland,2,POINT (1339319.999718883 904019.9999515081),0.0,1.0
4,4,7.103984,13.880937,3,138282925.0,1.0,Herbaceous/Grassland,2,POINT (1339319.999718883 904019.9999515081),0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
944,944,-5.424299,29.747131,3,138283004.0,80.0,Open water - freshwater,2,POINT (2870189.999993738 -690990.0000399862),0.0,0.0
945,945,-5.424299,29.747131,9,138283004.0,80.0,Open water - freshwater,2,POINT (2870189.999993738 -690990.0000399862),1.0,1.0
946,946,-5.424299,29.747131,10,138283004.0,80.0,Open water - freshwater,2,POINT (2870189.999993738 -690990.0000399862),NaN,NaN
947,947,-5.424299,29.747131,11,138283004.0,80.0,Open water - freshwater,2,POINT (2870189.999993738 -690990.0000399862),NaN,NaN


In [35]:
cross_Meghan = cross_val.merge(new_wofs, on=['lat', 'month'])
cross_Meghan[:3]

,Unnamed: 0_x,lat,lon_x,month,waterflag,waterflagAFR,waterflagAFR.1,waterflagAGR,waterflagAGR.1,waterflagRCM,...,agree_percent,Unnamed: 0_y,lon_y,plot_id,pl_plotid,class,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION
0,60,-25.97481,29.6125,5,1.0,NaN,1.0,1.0,1.0,NaN,...,85.714286,57,29.6125,139368554.0,6.0,Open water - freshwater,1,0.0,2.0,0
1,62,-25.97481,29.6125,7,1.0,1.0,1.0,1.0,1.0,NaN,...,100.000000,58,29.6125,139368554.0,6.0,Open water - freshwater,1,0.0,1.0,0
2,64,-25.97481,29.6125,9,1.0,1.0,1.0,1.0,1.0,NaN,...,100.000000,59,29.6125,139368554.0,6.0,Open water - freshwater,1,1.0,1.0,1
